In [8]:
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from joblib import dump, load

# Processamento dos dados de entrada

É criada um dataframe contendo um "bloco" por linha onde cada bloco possui um tipo. As colunas contém o texto do bloco e o tipo de emenda do bloco.

In [9]:
# Processamento dos Dados
path = "../data/tag_files"


tagList = ["I-","E-","B-"]
files = []
for dirpath, dirnames, filenames in os.walk(path):
    for filename in filenames:
        files.append(os.path.normpath(os.path.join(dirpath,filename)))

emendas = pd.DataFrame(columns = ['text','emdType'])
tupEmd = []


for file in files:
    with open(file, encoding = "utf-8") as f:
        emdTxt = []
        previousType = None
        for line in f.readlines():
            token,emdType = line.split()
            if any(x in emdType for x in tagList):
                emdType = emdType[2:]
            if previousType != emdType and previousType != None:
                tupEmd.append([" ".join(emdTxt), previousType])
                emdTxt = []
                
            emdTxt.append(token)
            previousType = emdType
        tupEmd.append([" ".join(emdTxt), emdType])


for index in range(len(tupEmd)):
    emendas.loc[index,'text'] = tupEmd[index][0]
    emendas.loc[index,'emdType'] = tupEmd[index][1]

# Classificador de Emendas

In [10]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(emendas['text'])
y = emendas['emdType']

## Separação das Bases

Separa as bases em treinamento e teste. A base de treinamento é usada na criação do modelo e a base de teste é usada para validação do modelo posteriormente.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Oversampling

Foi feito oversampling para gerar novos dados de emendas ADD pois haviam muito poucos casos na base de treino, o que gerava uma baixa acurácia especialmente para essa classe.

In [12]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X, y)

## Criação do Modelo

Cria o modelo e mostra a acurácia média obtida

In [13]:
clf = SVC(gamma = 'auto')
clf.fit(X_resampled,y_resampled)

print(clf.score(X_test,y_test))

0.8283582089552238


## Salvando o modelo treinado


O modelo é exportado juntamente com o vectorizer num objeto Pipeline para posterior uso.

In [24]:
from joblib import dump,load

pipeline = Pipeline([('vectorizer', vectorizer), ('clf_emend', clf)])
dump(pipeline,"emend_clf_pipe")

['emend_clf_pipe']

## Predição e Matriz de Confusão

Matriz de confusão mostra nas colunas o tipo da emenda e na linha a previsão feita para o tipo da emenda. Na diagonal portanto temos a quantidade de acertos para cara tipo de emenda.

In [25]:
y_pred = clf.predict(X_test)

cm = confusion_matrix(y_pred,y_test, labels = ['ADD','MOD','SUP','O'])
print(cm)

[[ 7  4  0  0]
 [ 1 23  0  0]
 [ 7 10 12  0]
 [ 0  0  1 69]]


## Acurácia das Classes

Acurácia separada por classe

In [26]:
cmAcc = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

print(cmAcc.diagonal())

[0.63636364 0.95833333 0.4137931  0.98571429]


In [27]:
cmDf = pd.DataFrame(cm,columns = ['ADD','MOD','SUP','O'])
cmDf['types'] = ['ADD','MOD','SUP','O']

In [28]:
cols = cmDf.columns.tolist()
cols.insert(0,cols.pop(4))
cmDf = cmDf[cols]
cmDf['Accuracy'] = cmAcc.diagonal()

In [29]:
cmDf.head()

,types,ADD,MOD,SUP,O,Accuracy
0,ADD,7,4,0,0,0.636364
1,MOD,1,23,0,0,0.958333
2,SUP,7,10,12,0,0.413793
3,O,0,0,1,69,0.985714


## Output resultados

Cria um arquivo .csv para avaliação dos resultados do modelo

In [20]:
cmDf.to_csv('../data/results/modelo_classificador_de_emendas/emdendasPredResultados.csv')

# Output de Emendas

Cria um arquivo .csv contendo o tipo de emenda e o tipo previso da emenda.

In [21]:
addDF = emendas.copy()
addDF = addDF.iloc[y_test.index]
addDF['emdTypePred'] = y_pred
addDF.head()

,text,emdType,emdTypePred
70,MPV 870 00040 CONGRESSO NACIONAL ETIQ UETA APR...,O,O
218,"Dê-se , ao artigo 65 da Medida Provisória nº 8...",MOD,MOD
258,"Os Arts . 37 , 38 , 43 e 44 , da Medida Provis...",MOD,MOD
33,Acrescente-se ao art . 37 da Medida Provisória...,ADD,SUP
42,MPV 870 00024 COMISSÃO MISTA DA MEDIDA PROVISÓ...,O,O


In [23]:
addDF.to_csv('../data/results/modelo_classificador_de_emendas/emdendasPred.csv')